# бейзлайны:

In [1]:
import pandas as pd
import numpy as np
import torch
import pickle

from metrics import ( 
    calculate_grouped_ndcg_random, 
    calculate_grouped_ndcg_sum_popularity,
    calculate_grouped_ndcg_with_embeddings,
    calculate_grouped_ndcg_for_bert4rec_output
)

In [2]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('movielens_1m_dataset/ratings.dat', sep='::', header=None, names=rnames, engine='python', encoding='ISO-8859-1')

In [3]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [4]:
ratings.sort_values(["user_id", "timestamp"], inplace=True)

ratings = ratings[["user_id", "movie_id", "rating"]]

ratings.reset_index(drop=True, inplace=True)

ratings = ratings.astype(int)

ratings

,user_id,movie_id,rating
0,1,3186,4
1,1,1270,5
2,1,1721,4
3,1,1022,5
4,1,2340,3
...,...,...,...
1000204,6040,2917,4
1000205,6040,1921,4
1000206,6040,1784,3
1000207,6040,161,3


In [5]:
ratings.user_id.nunique(), ratings.user_id.min(), ratings.user_id.max()

(6040, 1, 6040)

In [6]:
ratings.movie_id.nunique(), ratings.movie_id.min(), ratings.movie_id.max()

(3706, 1, 3952)

In [7]:
ratings.rating = 1

ratings

,user_id,movie_id,rating
0,1,3186,1
1,1,1270,1
2,1,1721,1
3,1,1022,1
4,1,2340,1
...,...,...,...
1000204,6040,2917,1
1000205,6040,1921,1
1000206,6040,1784,1
1000207,6040,161,1


In [8]:
# ratings = ratings[ratings.groupby("movie_id")['user_id'].transform('count') >= 5]
# дабы подстроиться под берт. потом вернем
ratings = ratings[ratings.groupby("user_id")['movie_id'].transform('count') >= 5]
ratings = ratings[ratings.groupby("movie_id")['user_id'].transform('count') >= 0]

ratings.reset_index(drop=True, inplace=True)

ratings

,user_id,movie_id,rating
0,1,3186,1
1,1,1270,1
2,1,1721,1
3,1,1022,1
4,1,2340,1
...,...,...,...
1000204,6040,2917,1
1000205,6040,1921,1
1000206,6040,1784,1
1000207,6040,161,1


In [9]:
print("statistics:", ratings.user_id.nunique(), ratings.movie_id.nunique())

statistics: 6040 3706


In [10]:
def create_index_mapping(df):
    """
    Создает словарь, отображающий айдишники юзеров и айтемов в
    небольшие различные числа (соответствующие индексам в enumerate)
    Зачем это нужно: implicit ALS хочет спарс матрицу для обучения, а размер scipy спарс
    матрицы зависит от максимального значения айдишника. Применение такого отображения к данным
    позволит работать с матрицами значительно меньших размеров.
    Parameters
        df: датасет, с колонок которого создаются словари
    Returns
        u2ix, i2ix: вышеописанные словари
    """
    u2ix = {user_id: i for i, user_id in enumerate(df.user_id.unique())}
    i2ix = {item_id: i for i, item_id in enumerate(df.movie_id.unique())}
    return u2ix, i2ix

def apply_index_mapping(df, u2ix, i2ix) -> pd.DataFrame:
    """
    Применяет вышеописанные словари к колонкам "user_id" и "movie_id".
    Parameters
        df: датасет, к которому применяется маппинг по словарям
        u2ix: словарь "user_id": "небольшое число"
        i2ix: словарь "movie_id": "небольшое число"
    Returns
        df: датасет с измененными (согласно u2ix и i2ix) значениями в колонках
    """
    df.user_id = df.user_id.map(lambda x: u2ix[x])
    df.movie_id = df.movie_id.map(lambda x: i2ix[x])
    return df

u2ix, i2ix = create_index_mapping(ratings)
ratings = apply_index_mapping(ratings, u2ix, i2ix)

leave-last-out:

In [11]:
test = ratings.groupby("user_id").tail(1)

test

,user_id,movie_id,rating
52,0,52,1
181,1,174,1
232,2,207,1
253,3,88,1
451,4,384,1
...,...,...,...
999522,6035,927,1
999724,6036,684,1
999744,6037,1587,1
999867,6038,579,1


In [12]:
train_and_val = ratings.drop(test.index, axis=0)

train_and_val

,user_id,movie_id,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1
...,...,...,...
1000203,6039,1097,1
1000204,6039,1248,1
1000205,6039,370,1
1000206,6039,89,1


In [13]:
val = train_and_val.groupby("user_id").tail(1)

val

,user_id,movie_id,rating
51,0,51,1
180,1,173,1
231,2,206,1
252,3,217,1
450,4,383,1
...,...,...,...
999521,6035,3356,1
999723,6036,244,1
999743,6037,252,1
999866,6038,562,1


In [14]:
train = train_and_val.drop(val.index, axis=0)

train

,user_id,movie_id,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1
...,...,...,...
1000202,6039,180,1
1000203,6039,1097,1
1000204,6039,1248,1
1000205,6039,370,1


In [15]:
train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

train_and_val.reset_index(drop=True, inplace=True) # нужно только для бейзлайнов и алс, мы там ниче не смотрим на val и не тюним

In [16]:
# так, берту не нужно время, ток порядок
train_dict = dict(train.groupby('user_id').apply(lambda d: list(d['movie_id'])))
val_dict = dict(val.groupby('user_id').apply(lambda d: list(d['movie_id'])))
test_dict = dict(test.groupby('user_id').apply(lambda d: list(d['movie_id'])))

In [17]:
dataset_for_bert4rec = {'train': train_dict,
                       'val': val_dict,
                       'test': test_dict,
                       'umap': u2ix,
                       'smap': i2ix}

In [18]:
import pickle

with open('prepared_dataset/dataset_for_bert4rec.pickle', 'wb') as handle:
    pickle.dump(dataset_for_bert4rec, handle, protocol=pickle.HIGHEST_PROTOCOL)

... продолжаем:

In [19]:
train_and_val

,user_id,movie_id,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1
...,...,...,...
994164,6039,1097,1
994165,6039,1248,1
994166,6039,370,1
994167,6039,89,1


In [20]:
test

,user_id,movie_id,rating
0,0,52,1
1,1,174,1
2,2,207,1
3,3,88,1
4,4,384,1
...,...,...,...
6035,6035,927,1
6036,6036,684,1
6037,6037,1587,1
6038,6038,579,1


In [21]:
user_positively_interacted_with = ratings.groupby('user_id')['movie_id'].apply(set).to_dict()

In [22]:
assert len(user_positively_interacted_with[1]) == len(train_and_val[train_and_val.user_id == 1]) + 1

In [23]:
item_count = ratings["movie_id"].value_counts().to_dict()
item_probabilities = {k: v / sum([x for x in item_count.values()]) for k, v in item_count.items()}

In [24]:
item_probabilities[62]

0.003427283697707179

In [25]:
item_probabilities[731]

0.00035592561154718663

In [26]:
import numpy as np
from numpy.random import choice

negative_samples = dict()
test_rows = []

for _, row in test.iterrows():
    test_interactions = [(row["user_id"], row["movie_id"], row["rating"],)]
    
    np.random.seed(row["user_id"])
    negative_sampled_interactions = list(choice(list(item_probabilities.keys()), 800, replace=False, p=list(item_probabilities.values())))
    negative_sampled_interactions = [x for x in negative_sampled_interactions if x not in user_positively_interacted_with[row["user_id"]]]
    negative_sampled_interactions = negative_sampled_interactions[:100]
    
    # потом, когда будем делать берт, будем ссылаться на сгенерированные здесь негативы:
    negative_samples[row["user_id"]] = negative_sampled_interactions

    test_interactions.extend([(row["user_id"], x, 0,) for x in negative_sampled_interactions])
    
    test_rows.extend(test_interactions)
    
test_rows[:10]

[(0, 52, 1),
 (0, 1068, 0),
 (0, 731, 0),
 (0, 1026, 0),
 (0, 314, 0),
 (0, 1148, 0),
 (0, 1008, 0),
 (0, 1030, 0),
 (0, 1513, 0),
 (0, 2575, 0)]

In [27]:
negative_samples[0][:9]

[1068, 731, 1026, 314, 1148, 1008, 1030, 1513, 2575]

In [28]:
neg_sampled_test = pd.DataFrame(test_rows, columns=["user_id", "movie_id", "rating"])

neg_sampled_test

,user_id,movie_id,rating
0,0,52,1
1,0,1068,0
2,0,731,0
3,0,1026,0
4,0,314,0
...,...,...,...
610035,6039,442,0
610036,6039,581,0
610037,6039,147,0
610038,6039,1327,0


### random baseline:

In [29]:
ndcgresults = []
for i in range(10):
    ndcg_value = calculate_grouped_ndcg_random(train_and_val, neg_sampled_test, 10, i)
    
    ndcgresults.append(ndcg_value)
    
np.mean(ndcgresults), np.std(ndcgresults)

(0.04438110400529842, 0.001780072420999918)

### pop baseline:

In [30]:
calculate_grouped_ndcg_sum_popularity(train_and_val, neg_sampled_test, 10)

0.06391453558404456

### iALS baseline:

In [31]:
import scipy.sparse as sparse


def create_sparse_matrix(df: pd.DataFrame) -> sparse.csr_matrix:
    """
    Делает разреженную матрицу из пандас датафрейма. Нужно для
    обучения implicit.als.AlternatingLeastSquares.
    Parameters
        df (pd.DataFrame): датасет
    Returns
        csr (sparse.csr_matrix): разреженная матрица
    """
    csr = sparse.csr_matrix((df.rating, (df.user_id, df.movie_id)))
    return csr

sparse_train_and_val = create_sparse_matrix(train_and_val)

In [32]:
import implicit

In [33]:
iALS = implicit.als.AlternatingLeastSquares(factors=100, iterations=150)

C:\Users\atama\anaconda3\envs\conda_bert4rec\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [34]:
iALS.fit(sparse_train_and_val)

  0%|          | 0/150 [00:00<?, ?it/s]

In [36]:
calculate_grouped_ndcg_with_embeddings(neg_sampled_test, iALS, 10)

0.22401902697663326

In [37]:
BPRMF = implicit.bpr.BayesianPersonalizedRanking(factors=60, iterations=125)

In [38]:
BPRMF.fit(sparse_train_and_val)

  0%|          | 0/125 [00:00<?, ?it/s]

In [39]:
calculate_grouped_ndcg_with_embeddings(neg_sampled_test, BPRMF, 10)

0.26978218387643793

# начинаем берт4рек:

In [44]:
import argparse

import pandas as pd
import pickle
import random
import torch

from tqdm import trange
from collections import Counter

import numpy as np
from numpy.random import choice

In [45]:
# fix argparse in ipython
import sys
sys.argv = ['']

In [46]:
from bert4rec_modules_and_configs.utils import *
from bert4rec_modules_and_configs.options import args

In [47]:
# то что сохранили

def read_data(prepared_data_path):
    with open(prepared_data_path, 'rb') as handle:
        dataset = pickle.load(handle)
    return dataset

In [48]:
data = read_data(args.prepared_data_path)

train_data = data['train']
val_data = data['val']
test_data = data['test']
umap = data['umap']
smap = data['smap']

готовим трейн датасет и даталоадер:

In [49]:
class BertTrainDataset(torch.utils.data.Dataset):
    def __init__(self, u2seq, max_len, mask_prob, mask_token, num_items, rng):
        self.u2seq = u2seq
        self.users = sorted(self.u2seq.keys())  # дополнительный "надежный" (с заданным порядком) способ ходить по train_data, в отличие от хождения по train_data по ключу
        self.max_len = max_len
        self.mask_prob = mask_prob
        print("self.mask_prob", self.mask_prob)
        self.mask_token = mask_token
        print("self.mask_token", self.mask_token)
        self.num_items = num_items
        self.rng = rng

    def __len__(self):
        return len(self.users)

    def __getitem__(self, index):
        user = self.users[index]
        seq = self._getseq(user)

        tokens = []
        labels = []
        for s in seq: # касательно каждого s заполняем токен и лейбл
            prob = self.rng.random()  # сгенерили для конкретного s вероятность uniform, deterministic
            if prob < self.mask_prob:  # если prob меньше self.mask_prob == 0.15: TLDR С ВЕРОЯТНОСТЬЮ 80% ЗАПОЛНИМ ТОКЕН МАСКТОКЕНОМ, А НЕ ЭТИМ ITEM INDEX
                prob /= self.mask_prob  # то сильно бустим, затем...

                # if prob < 0.8:
                if prob < 0.99:
                    tokens.append(self.mask_token)  # если оч слабо то заполняем токен масктокеном - который max(item !INDEX!) + 1 или self.item_count + 1
                # if prob < 0.9:
                elif prob < 0.999:
                    tokens.append(self.rng.randint(1, self.num_items))  # если попал в маленькое окошко - то рандомно между 1 и self.item_count (== self.num_items) НАХУЯ?
                else:
                    tokens.append(s) # emergency(?) вариант, но лэйбл всё равно заполнится ненормальным значением - он заполнится ЭТИМ ITEM INDEX

                labels.append(s)
            else:
                tokens.append(s)
                labels.append(0)  # !!!ЭТО СТРАННО ВЕДЬ СУЩНОСТЬ S [0, MAX INDEX]!!!

        # в итоге каждому s присвоится либо (s, 0) если не замаскирован, либо (mask_token<под вопросом>, s). окей, для чего? !!!ЭТО СТРАННО ВЕДЬ СУЩНОСТЬ S [0, MAX INDEX]!!!

        # видимо max_len это не длина окна подпоследовательности из всей последовательности (если использование такой подпоследовательности ваще в этой реализации будет как-то фигурировать), а то, раньше чего мы 100% забываем
        tokens = tokens[-self.max_len:]
        labels = labels[-self.max_len:]
        # был 140 - стал 100
        # был 90 - стал 90

        # тупа паддинг, если изначальный seq был меньше max_len
        mask_len = self.max_len - len(tokens)

        # прилепляем нули слева. почему нули???? !!!ЭТО СТРАННО ВЕДЬ СУЩНОСТЬ S [0, MAX INDEX]!!!
        tokens = [0] * mask_len + tokens
        labels = [0] * mask_len + labels

        # лол ща попробуем маскировать с 10% вероятностью последний айтем
        prob = self.rng.random()
        if prob < 0.1:
            tokens[-1] = self.mask_token
            labels[-1] = seq[-1]

        # в итоге каждому s присвоится либо (s, 0) если не замаскирован, либо (mask_token<под вопросом><уже не под вопросом>, s) + в начале будут прилеплены нули, если не дотягивает до max_len, или иначе обрублено [-max_len:]. окей, для чего? !!!ЭТО СТРАННО ВЕДЬ СУЩНОСТЬ S [0, MAX INDEX]!!!
        # tokens    0   0   0   2969, 1574,   957,    1178,   <3707>, 1658,   <3707>, 1117
        # labels    0   0   0   0     0       0       0       2147    0       3177    0
        return torch.LongTensor(tokens), torch.LongTensor(labels)

        # КАК ЕЩЁ ЧАСТО ДЕЛАЕТСЯ И КАК ДЕЛАЛОСЬ В СТАТЬЕ: БЕРЕТСЯ СЛУЧАЙНО 10% ЮЗЕРСКИХ ПОСЛЕДОВАТЕЛЬНОСТЕЙ ИЗ ТРЕЙНА И ДЛЯ НИХ ТОЛЬКО ПОСЛЕДНИЙ ТОКЕН ЗАМЕНЯЕТСЯ МАСКОЙ. мы так не делаем, в представленной реализации которую я разбираю такого случайного выбора 10% из трейна (для навешивания маски только на конец) нет.
        # теперь я тоже так делаю
        
    def _getseq(self, user):
        return self.u2seq[user]

In [50]:
train_torch_dataset = BertTrainDataset(
    u2seq=      train_data,
    max_len=    args.bert_max_len,
    mask_prob=  args.bert_mask_prob,
    mask_token= len(smap) + 1,
    num_items=  len(smap),
    rng=        random.Random(args.dataloader_random_seed)
)

self.mask_prob 0.15
self.mask_token 3707


In [51]:
train_torch_dataloader = torch.utils.data.DataLoader(
    dataset=train_torch_dataset,
    batch_size=args.train_batch_size,
    shuffle=True,
    pin_memory=True
)

In [52]:
# <показать здесь что получится для такого-то юзера>

готовим валидационные и тестовые датасет и даталоадер:

прежде чем сделать эти датасеты, сгенерируем негативы для юзеров, чтобы у них был не только один true positive на отложенной выборке, а ещё сотня негативов - тогда сможем замерить валидац метрики. семплируем по популярности (согласно статье)

In [53]:
# popularity = Counter()
# for user in range(len(umap)):
#     popularity.update(train_data[user])
#     popularity.update(val_data[user])
#     popularity.update(test_data[user])
# item_probabilities = {k: v / sum([x for x in popularity.values()]) for k, v in popularity.items()}

# print("PROB CHECK")
# print(item_probabilities[62])
# print(item_probabilities[731])

# negative_samples = {}
# print('Sampling negative items')
# for user in trange(len(umap)):
#     seen = set(train_data[user])
#     seen.update(val_data[user])
#     seen.update(test_data[user])

#     np.random.seed(user)

#     negative_sampled_interactions = list(choice(list(item_probabilities.keys()), 800, replace=False, p=list(item_probabilities.values())))
#     negative_sampled_interactions = [x for x in negative_sampled_interactions if x not in seen]
#     negative_sampled_interactions = negative_sampled_interactions[:100]

#     negative_samples[user] = negative_sampled_interactions
###

# взяли negative_samples, сделанные на этапе построения бейзлайнов, для 100% идентичности замера ndcg@10.

print("CHECK")
print(negative_samples[0][:9])

CHECK
[1068, 731, 1026, 314, 1148, 1008, 1030, 1513, 2575]


In [54]:
class BertEvalDataset(torch.utils.data.Dataset):
    def __init__(self, u2seq, u2answer, max_len, mask_token, negative_samples):
        self.u2seq = u2seq
        self.users = sorted(self.u2seq.keys()) # дополнительный "надежный" (с заданным порядком) способ ходить по train_data, в отличие от хождения по train_data по ключу
        self.u2answer = u2answer
        self.max_len = max_len
        self.mask_token = mask_token
        self.negative_samples = negative_samples

    def __len__(self):
        return len(self.users)

    def __getitem__(self, index):
        user = self.users[index]
        seq = self.u2seq[user]
        answer = self.u2answer[user]
        negs = self.negative_samples[user]

        candidates = answer + negs  # ну понятно, [228] + [142, 1488, 0, ...]
        labels = [1] * len(answer) + [0] * len(negs)  # сказали что первое позитив, остальное негативы

        seq = seq + [self.mask_token]  # прилепили <3707> в конец. теперь хорошо обученная модель сможет угадать его label

        seq = seq[-self.max_len:]  # эти три строки - всё так же как в трейне
        padding_len = self.max_len - len(seq)
        seq = [0] * padding_len + seq

        return torch.LongTensor(seq), torch.LongTensor(candidates), torch.LongTensor(labels)
        # например:
        # seq (в сущности как tokens в трейне)  0   0   2969,   1574,   957,    1178,   2147,   1658,   3177,   1117, <3707>
        #
        # candidates: [индекс айтема реального позитива; сгенерированные негативы]
        # labels:     [1, 0, 0, 0, 0, ...]

In [55]:
val_torch_dataset = BertEvalDataset(
    u2seq=              train_data,
    u2answer=           val_data,
    max_len=            args.bert_max_len,
    mask_token=         len(smap) + 1,
    negative_samples=   negative_samples
)

In [56]:
val_torch_dataloader = torch.utils.data.DataLoader(val_torch_dataset, batch_size=args.val_batch_size,
                                       shuffle=False, pin_memory=True)

In [57]:
test_torch_dataset = BertEvalDataset(
        u2seq=              train_data,
        u2answer=           test_data,
        max_len=            args.bert_max_len,
        mask_token=         len(smap) + 1,
        negative_samples=   negative_samples
    )

In [58]:
test_torch_dataloader = torch.utils.data.DataLoader(test_torch_dataset, batch_size=args.test_batch_size,
                                                       shuffle=False, pin_memory=True)

In [59]:
args.num_items = len(smap)

описываем модель:

In [62]:
from bert4rec_modules_and_configs.models.bert import BERTModel

In [63]:
model = BERTModel(args)

In [64]:
some_batch = next(iter(train_torch_dataloader))
seq, _ = some_batch
print(seq.shape)
print(seq[0, ...].unsqueeze(0).shape)
print(model(seq[0, ...].unsqueeze(0)).shape)

torch.Size([128, 100])
torch.Size([1, 100])
torch.Size([1, 100, 3707])


In [67]:
from bert4rec_modules_and_configs.trainers import trainer_factory

In [68]:
export_root = setup_train(args)

trainer = trainer_factory(args, model, train_torch_dataloader, val_torch_dataloader, test_torch_dataloader, export_root)

Folder created: C:\Users\atama\Documents\GitHub\bert4rec_ysda_seminar\seminar\experiments\test_2024-03-09_0
experiments\test_2024-03-09_0
experiments\test_2024-03-09_0\config.json

Namespace(mode='train', template=None, test_model_path=None, dataset_code='ml-1m', min_rating=0, min_uc=5, min_sc=0, split='leave_one_out', dataset_split_seed=98765, eval_set_size=500, prepared_data_path='prepared_dataset/dataset_for_bert4rec.pickle', dataloader_code='bert', dataloader_random_seed=0.0, train_batch_size=128, val_batch_size=128, test_batch_size=128, train_negative_sampler_code='random', train_negative_sample_size=0, train_negative_sampling_seed=0, test_negative_sampler_code='popular', test_negative_sample_size=100, test_negative_sampling_seed=98765, trainer_code='bert', device='cuda', num_gpu=1, device_idx='0', optimizer='Adam', lr=0.001, weight_decay=0, momentum=None, decay_step=25, gamma=1.0, num_epochs=100, log_period_as_iter=12800, metric_ks=[1, 5, 10, 20, 50, 100], best_metric='NDCG@10', 

In [69]:
trainer.train()

Val: N@1 0.007, N@5 0.028, N@10 0.045, R@1 0.007, R@5 0.050, R@10 0.102: 100%|██████████| 48/48 [00:04<00:00, 10.12it/s]
C:\Users\atama\anaconda3\envs\conda_bert4rec\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Update Best NDCG@10 Model at 1


Epoch 1, loss 7.838 : 100%|██████████| 48/48 [00:02<00:00, 18.27it/s]
Val: N@1 0.019, N@5 0.049, N@10 0.072, R@1 0.019, R@5 0.081, R@10 0.154: 100%|██████████| 48/48 [00:02<00:00, 21.35it/s]


Update Best NDCG@10 Model at 1


Epoch 2, loss 7.517 : 100%|██████████| 48/48 [00:02<00:00, 18.48it/s]
Val: N@1 0.023, N@5 0.064, N@10 0.086, R@1 0.023, R@5 0.104, R@10 0.175: 100%|██████████| 48/48 [00:02<00:00, 21.41it/s]


...

Epoch 73, loss 5.125 : 100%|██████████| 48/48 [00:02<00:00, 18.59it/s]
Val: N@1 0.264, N@5 0.417, N@10 0.457, R@1 0.264, R@5 0.557, R@10 0.681: 100%|██████████| 48/48 [00:02<00:00, 20.58it/s]
Epoch 74, loss 5.125 : 100%|██████████| 48/48 [00:02<00:00, 18.33it/s]
Val: N@1 0.267, N@5 0.420, N@10 0.461, R@1 0.267, R@5 0.558, R@10 0.683: 100%|██████████| 48/48 [00:02<00:00, 20.62it/s]


Update Best NDCG@10 Model at 74


Epoch 75, loss 5.118 : 100%|██████████| 48/48 [00:02<00:00, 17.96it/s]
Val: N@1 0.266, N@5 0.414, N@10 0.456, R@1 0.266, R@5 0.550, R@10 0.680: 100%|██████████| 48/48 [00:02<00:00, 21.05it/s]
Epoch 76, loss 5.123 : 100%|██████████| 48/48 [00:02<00:00, 18.15it/s]
Val: N@1 0.265, N@5 0.416, N@10 0.457, R@1 0.265, R@5 0.553, R@10 0.679: 100%|██████████| 48/48 [00:02<00:00, 20.83it/s]
Epoch 77, loss 5.091 : 100%|██████████| 48/48 [00:02<00:00, 18.48it/s] 
Val: N@1 0.264, N@5 0.417, N@10 0.455, R@1 0.264, R@5 0.558, R@10 0.675: 100%|██████████| 48/48 [00:02<00:00, 21.09it/s]
Epoch 78, loss 5.096 : 100%|██████████| 48/48 [00:02<00:00, 18.44it/s]
Val: N@1 0.262, N@5 0.416, N@10 0.456, R@1 0.262, R@5 0.555, R@10 0.678: 100%|██████████| 48/48 [00:02<00:00, 21.55it/s]
Epoch 79, loss 5.087 : 100%|██████████| 48/48 [00:02<00:00, 18.35it/s] 
Val: N@1 0.272, N@5 0.421, N@10 0.462, R@1 0.272, R@5 0.557, R@10 0.682: 100%|██████████| 48/48 [00:02<00:00, 21.28it/s]


Update Best NDCG@10 Model at 79


Epoch 80, loss 5.077 : 100%|██████████| 48/48 [00:02<00:00, 18.52it/s]
Val: N@1 0.265, N@5 0.419, N@10 0.459, R@1 0.265, R@5 0.560, R@10 0.683: 100%|██████████| 48/48 [00:02<00:00, 21.42it/s]
Epoch 81, loss 5.082 : 100%|██████████| 48/48 [00:02<00:00, 18.43it/s] 
Val: N@1 0.258, N@5 0.411, N@10 0.452, R@1 0.258, R@5 0.550, R@10 0.676: 100%|██████████| 48/48 [00:02<00:00, 21.49it/s]
Epoch 82, loss 5.072 : 100%|██████████| 48/48 [00:02<00:00, 18.44it/s]
Val: N@1 0.266, N@5 0.420, N@10 0.460, R@1 0.266, R@5 0.561, R@10 0.685: 100%|██████████| 48/48 [00:02<00:00, 21.19it/s]
Epoch 83, loss 5.072 : 100%|██████████| 48/48 [00:02<00:00, 18.45it/s] 
Val: N@1 0.264, N@5 0.415, N@10 0.455, R@1 0.264, R@5 0.551, R@10 0.676: 100%|██████████| 48/48 [00:02<00:00, 21.31it/s]
Epoch 84, loss 5.055 : 100%|██████████| 48/48 [00:02<00:00, 18.05it/s]
Val: N@1 0.271, N@5 0.420, N@10 0.461, R@1 0.271, R@5 0.556, R@10 0.681: 100%|██████████| 48/48 [00:02<00:00, 20.83it/s]
Epoch 85, loss 5.052 : 100%|██████████

In [70]:
trainer.test()

Test best model with test set!


Val: N@1 0.216, N@5 0.358, N@10 0.396, R@1 0.216, R@5 0.486, R@10 0.603: 100%|██████████| 48/48 [00:02<00:00, 19.39it/s]

{'Recall@100': 0.9978841145833334, 'NDCG@100': 0.47592918202281, 'Recall@50': 0.8781467018028101, 'NDCG@50': 0.4565200960884492, 'Recall@20': 0.7212999140222868, 'NDCG@20': 0.4253871502975623, 'Recall@10': 0.6029188372194767, 'NDCG@10': 0.3955337864657243, 'Recall@5': 0.48605685805281, 'NDCG@5': 0.3577783592045307, 'Recall@1': 0.2160915800680717, 'NDCG@1': 0.2160915800680717}


---

посчитаем ndcg@10 как в бейзлайнах - по датафрейму neg_sampled_test и по той функции рассчёта, которая использовалась там:

In [71]:
best_model = torch.load(os.path.join(export_root, 'models', 'best_acc_model.pth')).get('model_state_dict')

In [72]:
model.load_state_dict(best_model)

<All keys matched successfully>

In [73]:
model.eval()

BERTModel(
  (bert): BERT(
    (embedding): BERTEmbedding(
      (token): TokenEmbedding(3708, 256, padding_idx=0)
      (position): PositionalEmbedding(
        (pe): Embedding(100, 256)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer_blocks): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadedAttention(
          (linear_layers): ModuleList(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): Linear(in_features=256, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=256, bias=True)
          )
          (output_linear): Linear(in_features=256, out_features=256, bias=True)
          (attention): Attention()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=256, out_features=1024, bias=True)
          (w_2): Linear(in_features=1024, out_features=256, bias=True)
          (dr

In [74]:
from tqdm import tqdm

# получаем предикты для mask_token на тестовой последовательности:
scores_for_mask_token_for_each_batch = []

with torch.no_grad():
    tqdm_dataloader = tqdm(test_torch_dataloader)
    
    for batch_idx, batch in enumerate(tqdm_dataloader):
        batch = [x.to('cuda') for x in batch]
        
        scores = model(batch[0])
        scores_for_mask_token = scores[:, -1, :]
        
        scores_for_mask_token_for_each_batch.append(scores_for_mask_token.cpu())

100%|██████████| 48/48 [00:00<00:00, 50.64it/s]


In [75]:
scores_for_mask_token = torch.cat(scores_for_mask_token_for_each_batch, dim=0)

scores_for_mask_token.shape

torch.Size([6040, 3707])

In [76]:
user_to_scores_of_items = dict()

for user_ix, scores in enumerate(scores_for_mask_token):
    user_to_scores_of_items[user_ix] = dict()
    
    for item_ix, score in enumerate(scores):
        user_to_scores_of_items[user_ix][item_ix] = score.item()

In [77]:
print(
    user_to_scores_of_items[0][49], # видел на трейне
    user_to_scores_of_items[0][50], # видел на трейне
    user_to_scores_of_items[0][51], # не видел на трейне (а стоило бы вернуть - мы его украли в валидационное взаимодействие)
    user_to_scores_of_items[0][52], # не видел на трейне
    user_to_scores_of_items[0][53], # не видел на трейне (и в реальности его не было в train + val + test)
    user_to_scores_of_items[0][54], # не видел на трейне (и в реальности его не было в train + val + test)
)

8.646805763244629 10.688972473144531 9.815849304199219 10.665151596069336 2.121732473373413 2.446187973022461


выглядит правдоподобно

In [78]:
from metrics import *

In [79]:
def calculate_grouped_ndcg_for_bert4rec_output(valDf, model_scores, k):
    data = valDf.copy()
    
    data["predicted_rating"] = data.apply(lambda row: model_scores[row["user_id"]][row["movie_id"]], axis=1)
    
    nonnull_users = set(data[data.rating > 0].user_id)
    data = data[data.user_id.isin(nonnull_users)]
    
    return np.mean(data.groupby("user_id").apply(lambda x: ndcg_score(x.rating, x.predicted_rating, k)))

In [80]:
calculate_grouped_ndcg_for_bert4rec_output(neg_sampled_test, user_to_scores_of_items, 10)

0.3939330098851488